In [ ]:
import numpy as np
import os 
import sys 
import matplotlib
from sklearn.manifold import MDS as classic_MDS
import time 
import scipy
import matplotlib.pyplot as mplt 
import pylab as PLT
import plotly
import plotly.tools as tls
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from itertools import product
from sklearn.neighbors import KNeighborsClassifier
from pprint import pprint
from sklearn.metrics.pairwise import euclidean_distances
plotly.offline.init_notebook_mode()
data_dir = './'
sys.path.append('./')
import bs_dev
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# Load the digtis from the given file
digits_path = os.path.join(data_dir, 'digits-labels.npz')
load_dic = np.load(digits_path)
digits_labels = load_dic['l']
digits_data = load_dic['d']

n_total_digits = digits_data.shape[1]
# just check one input datapoint and its corresponding label: 
indx = np.random.randint(n_total_digits)
print(n_total_digits)
print("Showing a digit: {}".format(digits_labels[indx]))
mplt.imshow(digits_data[:, indx].T.reshape(28, 28).T)

In [ ]:
def pairwise_euclidean(X):
    return euclidean_distances(X, X)

In [ ]:
n_total = 300
all_inds = np.random.choice(np.arange(digits_labels.shape[0]), 
                            size=n_total, replace=False)
X_all = digits_data[:, all_inds]
Y_all = digits_labels[all_inds]
D_goal = pairwise_euclidean(X_all.T)

In [ ]:
print(digits_data.shape)
print(digits_labels.shape)
n_total = 1000
max_iters = 20
all_inds = np.random.choice(np.arange(digits_labels.shape[0]), 
                            size=n_total, replace=False)
X_all = digits_data[:, all_inds]
Y_all = digits_labels[all_inds]
D_goal = pairwise_euclidean(X_all.T)

embedding_dims = [5, 30]
rads = [1.]
initial_probabilities = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
thresholds = [0., 0.1, 0.2, 0.3, 0.4]

emb_dim = 100 
mds_res = {'FS CSMDS':{}, 
           'RN CSMDS':{},
           'BS CSMDS':{}}

for rad in rads:
    pat_search_MDS_creator = bs_dev.MDS(n_components=emb_dim,
                                        starting_radius=rad,
                                        max_iter=max_iters,
                                        mode='full_search',
                                        prob_thresh=0.3,
                                        initial_prob=.8,
                                        a_bs=0.05,
                                        verbose=2,
                                        dissimilarity='precomputed')
    (x_low_rank,
     time_logger) = pat_search_MDS_creator.fit_transform(D_goal)
    mds_res['FS CSMDS'][rad] = time_logger 
    mds_res['RN CSMDS'][rad] = {}
    mds_res['BS CSMDS'][rad] = {}

    for initial_prob in initial_probabilities:
        pat_search_MDS_creator = bs_dev.MDS(n_components=emb_dim,
                                            starting_radius=rad,
                                            max_iter=max_iters,
                                            mode='randomized',
                                            prob_thresh=0.3,
                                            initial_prob=.7,
                                            a_bs=0.05,
                                            verbose=2,
                                            dissimilarity='precomputed')
        (x_low_rank,
         time_logger) = pat_search_MDS_creator.fit_transform(D_goal)
        mds_res['RN CSMDS'][rad][initial_prob] = time_logger 
        mds_res['BS CSMDS'][rad][initial_prob] = {}
        
        for prob_thresh in thresholds:
            pat_search_MDS_creator = bs_dev.MDS(n_components=emb_dim,
                                                starting_radius=rad,
                                                max_iter=max_iters,
                                                mode='bootstrapped',
                                                prob_thresh=prob_thresh,
                                                initial_prob=initial_prob,
                                                a_bs=0.05,
                                                verbose=2,
                                                dissimilarity='precomputed')
            (x_low_rank,
             time_logger) = pat_search_MDS_creator.fit_transform(D_goal)
            mds_res['BS CSMDS'][rad][initial_prob][prob_thresh] = time_logger
            
            
            print("==============Above was: r:{} init:{} thresh:{}==================="
                  "".format(rad, initial_prob, prob_thresh))
            
    fig, axes = PLT.subplots(len(initial_probabilities), 
                             len(thresholds), figsize=(18, 18), sharex=True, sharey=True)

    for i, i_pr in enumerate(initial_probabilities):
        for j, j_th in enumerate(thresholds):
            axes[0, j].set_title('Thresh '+str(j_th), fontsize=18)
#             print(np.array([x[0] for x in mds_res['FS CSMDS'][rad]]))
            axes[i, j].plot(np.array([y[0] for y in mds_res['FS CSMDS'][rad]]) - 
                            mds_res['FS CSMDS'][rad][0][0], 
                            np.array([x[-1] for x in mds_res['FS CSMDS'][rad]]))
            axes[i, j].plot(np.array([y[0] for y in mds_res['RN CSMDS'][rad][i_pr]]) - 
                            mds_res['RN CSMDS'][rad][i_pr][0][0], 
                            np.array([x[-1] for x in mds_res['RN CSMDS'][rad][i_pr]]))
            axes[i, j].plot(np.array([y[0] for y in mds_res['BS CSMDS'][rad][i_pr][j_th]]) - 
                            mds_res['BS CSMDS'][rad][i_pr][j_th][0][0], 
                            np.array([x[-1] for x in mds_res['BS CSMDS'][rad][i_pr][j_th]]))
        axes[i, 0].set_ylabel('Init '+str(i_pr), fontsize=18)
    t1 = fig.text(0.5, +0.08, 'Time (seconds)', ha='center', va='center', fontsize=24)
    t2 = fig.text(+0.05, 0.5, 'Stress Error', ha='center', va='center', rotation='vertical', fontsize=24)


    handles, labels = axes[0, 0].get_legend_handles_labels()
#     PLT.legend(['FS CSMDS','RN CSMDS','BS CSMDS'], 
#                          loc=0, fontsize=25)
    leg = axes[-1, -1].legend(['FS CSMDS','RN CSMDS','BS CSMDS'], 
               bbox_to_anchor=(0., 1.02, 5., .102), 
               loc=(-0.9, +75), ncol=3, mode="expand",
              fontsize=24)
#     leg.set_in_layout(False)
    fig.savefig('./images/perf_rad_'+str(rad)+'.pdf', bbox_extra_artists=(leg, t1, t2), 
                bbox_inches='tight', format='pdf')
#     fig.savefig('./images/perf_rad_'+str(rad)+'.pdf', format='pdf')
    fig.tight_layout()
    PLT.show()